# Model evaluation 
Part 1


In [2]:
import pandas as pd
import numpy as np

In [3]:
file_path = 'data/dataset.csv'
df = pd.read_csv(file_path, sep=';', header=0)

df.head()

,ShotType,Competition,PlayerType,Transition,TwoLegged,Movement,Angle,Distance
0,above head,U14,F,1,1,no,73.83,0.73
1,layup,U14,F,0,1,no,28.13,1.02
2,above head,U14,F,0,1,no,51.88,7.22
3,above head,U14,F,1,1,no,80.84,3.64
4,above head,U14,F,0,1,no,30.89,7.20


In [7]:
null_counts = df.isnull().sum()
print(null_counts)

ShotType       0
Competition    0
PlayerType     0
Transition     0
TwoLegged      0
Movement       0
Angle          0
Distance       0
dtype: int64


### Helper functions

In [ ]:
def log_loss(y_true, y_pred):
    """
    Compute the log loss (logarithmic loss) for binary classification.

    Parameters:
    - y_true: True labels (0 or 1).
    - y_pred: Predicted probabilities (between 0 and 1).

    Returns:
    - log_loss: The computed log loss.
    """
    # Clip predicted probabilities to avoid log(0) or log(1)
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    
    loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    return loss

In [ ]:
def accuracy_score(y_true, y_pred):
    """
    Compute the accuracy score for classification.

    Parameters:
    - y_true: True labels.
    - y_pred: Predicted labels.

    Returns:
    - accuracy: The computed accuracy score.
    """
    # Check if predictions match true labels
    correct = np.sum(y_true == y_pred)
    
    accuracy = correct / len(y_true)
    
    return accuracy